# Predicting the Success of Prospects in the NHL

The goal of this notebook is to predict the statistics of prospects in the NHL in their rookie years using several methods. An NHL player is in their rookie year the last year they are eligible for the Calder Trophy, meaning their rookie year happens if the player:
* is 26 or younger
* has played more than 25 games in a season
* has played less than 6 games in the previous 2 seasons

# Phase 1: Preprocess the Data

In [2]:
# import needed libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O

Convieniently, Kaggle provides a large csv file on players and statistics dating back to 1963 (including draft information)

In [3]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/nhldraft.csv")

In [4]:
df.head()

,id,year,overall_pick,team,player,nationality,position,age,to_year,amateur_team,...,points,plus_minus,penalties_minutes,goalie_games_played,goalie_wins,goalie_losses,goalie_ties_overtime,save_percentage,goals_against_average,point_shares
0,1,2022,1,Montreal Canadiens,Juraj Slafkovsky,SK,LW,18.0,NaN,TPS (Finland),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2022,2,New Jersey Devils,Simon Nemec,SK,D,18.0,NaN,HK Nitra (Slovakia),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2022,3,Arizona Coyotes,Logan Cooley,US,C,18.0,NaN,USA U-18 Development Team (USDP/USHL),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2022,4,Seattle Kraken,Shane Wright,CA,C,18.0,NaN,Kingston Frontenacs (OHL),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2022,5,Philadelphia Flyers,Cutter Gauthier,SE,LW,18.0,NaN,USA U-18 Development Team (USDP/USHL),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# for this scenario, we do not need the later columns
df = df.iloc[:, :10]
df.head()

,id,year,overall_pick,team,player,nationality,position,age,to_year,amateur_team
0,1,2022,1,Montreal Canadiens,Juraj Slafkovsky,SK,LW,18.0,NaN,TPS (Finland)
1,2,2022,2,New Jersey Devils,Simon Nemec,SK,D,18.0,NaN,HK Nitra (Slovakia)
2,3,2022,3,Arizona Coyotes,Logan Cooley,US,C,18.0,NaN,USA U-18 Development Team (USDP/USHL)
3,4,2022,4,Seattle Kraken,Shane Wright,CA,C,18.0,NaN,Kingston Frontenacs (OHL)
4,5,2022,5,Philadelphia Flyers,Cutter Gauthier,SE,LW,18.0,NaN,USA U-18 Development Team (USDP/USHL)


Next, I will replace the NaN with zeros because any NaN information (except with 'to_year') should be replaced with a zero if it is not specified.

In [6]:
df.iloc[:, 10:] = df.iloc[:, 10:].fillna(0)
df.head()

,id,year,overall_pick,team,player,nationality,position,age,to_year,amateur_team
0,1,2022,1,Montreal Canadiens,Juraj Slafkovsky,SK,LW,18.0,NaN,TPS (Finland)
1,2,2022,2,New Jersey Devils,Simon Nemec,SK,D,18.0,NaN,HK Nitra (Slovakia)
2,3,2022,3,Arizona Coyotes,Logan Cooley,US,C,18.0,NaN,USA U-18 Development Team (USDP/USHL)
3,4,2022,4,Seattle Kraken,Shane Wright,CA,C,18.0,NaN,Kingston Frontenacs (OHL)
4,5,2022,5,Philadelphia Flyers,Cutter Gauthier,SE,LW,18.0,NaN,USA U-18 Development Team (USDP/USHL)


Now we have to convert categorical data into numeric so the data can be regressed on. For players and goalies, the only non-numeric data is:
*     the team that drafted them (team)
*     their nationality
*     their position (obviously for goalies it has been taken care of already)
*     their amateur team

This also ensures that when the ANOVA F-test is performed, that we can include categorical data such as nationality and the drafted team. For the amateur team, the league will be extracted from the data and then label encoding will be used to convert the categorical data into numeric ones.

In [7]:
# change from amateur team to amateur league
import re
count = 0
for string in df['amateur_team']:
  res = re.findall(r'\(.*?\)', string)
  df['amateur_team'][count] = res[0].replace('(', '').replace(')', '')
  count = count + 1
df.head()

<ipython-input-7-ecf6b6a2b8c7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['amateur_team'][count] = res[0].replace('(', '').replace(')', '')


,id,year,overall_pick,team,player,nationality,position,age,to_year,amateur_team
0,1,2022,1,Montreal Canadiens,Juraj Slafkovsky,SK,LW,18.0,NaN,Finland
1,2,2022,2,New Jersey Devils,Simon Nemec,SK,D,18.0,NaN,Slovakia
2,3,2022,3,Arizona Coyotes,Logan Cooley,US,C,18.0,NaN,USDP/USHL
3,4,2022,4,Seattle Kraken,Shane Wright,CA,C,18.0,NaN,OHL
4,5,2022,5,Philadelphia Flyers,Cutter Gauthier,SE,LW,18.0,NaN,USDP/USHL


In [8]:
df = df.rename(columns = {'amateur_team': 'amateur_league'})
df.amateur_league.value_counts()

OHL            1852
WHL            1721
QMJHL          1240
USHL            452
WCHL            413
               ... 
High-IL           1
Nova Scotia       1
Germany 2         1
RMJHL             1
GWMHA             1
Name: amateur_league, Length: 191, dtype: int64

With the teams, we can convert the teams into placements using the read_html() function from pandas.

In [9]:
import time
# make a table to store the ranks of the teams
team_rank_tbl = pd.DataFrame()

#cycle through all the years from 1963-2022 for seasonal ranks
for year in range(1963,2023):
  #get the seasonal data from hockey-reference.com
  url = "https://www.hockey-reference.com/leagues/NHL_" + str(year) + "_standings.html"
  first_season = pd.DataFrame()
  #based on year, will have to take certain tables
  if year < 1968 or year==2021: #still original six and COVID year
    first_season = pd.read_html(url)[1]
  elif year == 2005: #lockout year
    continue
  else: # division into two conferences
    first_season = pd.read_html(url)[2]

  #format the dataframes to fit the criteria
  year_col = [year]*first_season.shape[0]
  first_season['year'] = year_col
  first_season.insert(1, "year", first_season.pop("year"))
  first_season.insert(2, "league_standing", first_season.pop("Rk"))
  first_season = first_season.iloc[:, 0:3]
  first_season = first_season.rename(columns = {"Unnamed: 1":"team"})

  #combine the new dataframe with the old one
  team_rank_tbl = pd.concat([first_season, team_rank_tbl], ignore_index=False, axis=0)

  #sleep to prevent HTTP Error 429 (Too Many Requests)
  time.sleep(3)
team_rank_tbl

,year,team,league_standing
0,2022,Florida Panthers,1
1,2022,Colorado Avalanche,2
2,2022,Carolina Hurricanes,3
3,2022,Toronto Maple Leafs,4
4,2022,Minnesota Wild,5
...,...,...,...
1,1963,Chicago Black Hawks,2
2,1963,Montreal Canadiens,3
3,1963,Detroit Red Wings,4
4,1963,New York Rangers,5


Now we join the team rank table with the draft table to then add the team's league standing into the table.

In [10]:
# join the two tables together
df = df.merge(team_rank_tbl, on=['team', 'year'])
df.insert(4, "league_standing", df.pop("league_standing"))
df.rename(columns={"league_standing":"drafted_team_standing"})

,id,year,overall_pick,team,drafted_team_standing,player,nationality,position,age,to_year,amateur_league
0,1,2022,1,Montreal Canadiens,32,Juraj Slafkovsky,SK,LW,18.0,NaN,Finland
1,26,2022,26,Montreal Canadiens,32,Filip Mesar,SK,RW,18.0,NaN,Slovakia
2,33,2022,33,Montreal Canadiens,32,Owen Beck,CA,C,18.0,NaN,OHL
3,62,2022,62,Montreal Canadiens,32,Lane Hutson,US,D,18.0,NaN,USDP/USHL
4,75,2022,75,Montreal Canadiens,32,Vinzenz Rohrer,AT,C,18.0,NaN,OHL
...,...,...,...,...,...,...,...,...,...,...,...
11426,12249,1963,20,New York Rangers,5,Campbell Alleson,CA,D,NaN,NaN,
11427,12235,1963,6,Toronto Maple Leafs,1,Walt McKechnie,CA,C,16.0,1983.0,WOJBHL
11428,12241,1963,12,Toronto Maple Leafs,1,Neil Clairmont,CA,LW,NaN,NaN,
11429,12246,1963,17,Toronto Maple Leafs,1,Jim McKenny,CA,D,16.0,1979.0,MetJAHL


For the amateur team, we can convert it into numerical data using Binary Encoding to allow every league of the 191 to be represented equally without drastically increasing the dimensionality of the dataset.

Binary Encoding assigns a number to the category, and then turns the number into its binary representation, making it very good to encode this feature, since it is a nominal feature (meaning there is no order between categories) and it has a lot of categories

In [11]:
# convert amateur team using Binary Encoding
!pip install category_encoders
import category_encoders as ce

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.9 MB/s eta 0:00:00


In [12]:
#instantiate encoder (specifically BinaryEncoder)
encoder = ce.BinaryEncoder(cols=['amateur_league'])
#fit and transform
ama_team_binary = encoder.fit_transform(df['amateur_league'])
ama_team_binary
#merge it with the dataframe and remove the 'amateur_league' column
temp = df
temp = pd.concat([temp.iloc[:, :10], ama_team_binary], axis=1)
df = temp
df

,id,year,overall_pick,team,league_standing,player,nationality,position,age,to_year,amateur_league_0,amateur_league_1,amateur_league_2,amateur_league_3,amateur_league_4,amateur_league_5,amateur_league_6,amateur_league_7
0,1,2022,1,Montreal Canadiens,32,Juraj Slafkovsky,SK,LW,18.0,NaN,0,0,0,0,0,0,0,1
1,26,2022,26,Montreal Canadiens,32,Filip Mesar,SK,RW,18.0,NaN,0,0,0,0,0,0,1,0
2,33,2022,33,Montreal Canadiens,32,Owen Beck,CA,C,18.0,NaN,0,0,0,0,0,0,1,1
3,62,2022,62,Montreal Canadiens,32,Lane Hutson,US,D,18.0,NaN,0,0,0,0,0,1,0,0
4,75,2022,75,Montreal Canadiens,32,Vinzenz Rohrer,AT,C,18.0,NaN,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,12249,1963,20,New York Rangers,5,Campbell Alleson,CA,D,NaN,NaN,0,1,0,0,1,0,1,0
11427,12235,1963,6,Toronto Maple Leafs,1,Walt McKechnie,CA,C,16.0,1983.0,1,0,0,1,0,0,1,1
11428,12241,1963,12,Toronto Maple Leafs,1,Neil Clairmont,CA,LW,NaN,NaN,0,1,0,0,1,0,1,0
11429,12246,1963,17,Toronto Maple Leafs,1,Jim McKenny,CA,D,16.0,1979.0,0,1,1,1,0,0,1,0


We can use the same process to convert the nationalities into numerical data

In [13]:
# find the number of unique values in the nationalities column
print("number of distinct values: " + str(len(df.nationality.value_counts())))
df.nationality.value_counts()

number of distinct values: 46


CA    5998
US    2484
SE     758
RU     693
FI     472
CZ     450
SK     151
DE      80
CH      71
LV      35
UA      30
BY      29
DK      24
KZ      24
NO      23
GB      20
AT      19
FR       8
SI       8
PL       8
YU       3
UZ       3
HU       3
LT       3
JP       2
KR       2
BE       2
BS       2
BR       2
NG       2
EE       2
SU       2
ZA       1
HT       1
TW       1
PY       1
BN       1
AU       1
TZ       1
JM       1
ME       1
IT       1
NL       1
CN       1
TH       1
VE       1
Name: nationality, dtype: int64

In [14]:
# convert the nationalities to numerical data using BinaryEncoder
country_encoder = ce.BinaryEncoder(cols=['nationality'])
#fit and transform
country_bin = country_encoder.fit_transform(df['nationality'])
country_bin

temp = df
temp = pd.concat([temp, country_bin], axis=1)
df = temp
df = df.drop('nationality', axis=1)
df

,id,year,overall_pick,team,league_standing,player,position,age,to_year,amateur_league_0,...,amateur_league_4,amateur_league_5,amateur_league_6,amateur_league_7,nationality_0,nationality_1,nationality_2,nationality_3,nationality_4,nationality_5
0,1,2022,1,Montreal Canadiens,32,Juraj Slafkovsky,LW,18.0,NaN,0,...,0,0,0,1,0,0,0,0,0,1
1,26,2022,26,Montreal Canadiens,32,Filip Mesar,RW,18.0,NaN,0,...,0,0,1,0,0,0,0,0,0,1
2,33,2022,33,Montreal Canadiens,32,Owen Beck,C,18.0,NaN,0,...,0,0,1,1,0,0,0,0,1,0
3,62,2022,62,Montreal Canadiens,32,Lane Hutson,D,18.0,NaN,0,...,0,1,0,0,0,0,0,0,1,1
4,75,2022,75,Montreal Canadiens,32,Vinzenz Rohrer,C,18.0,NaN,0,...,0,0,1,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,12249,1963,20,New York Rangers,5,Campbell Alleson,D,NaN,NaN,0,...,1,0,1,0,0,0,0,0,1,0
11427,12235,1963,6,Toronto Maple Leafs,1,Walt McKechnie,C,16.0,1983.0,1,...,0,0,1,1,0,0,0,0,1,0
11428,12241,1963,12,Toronto Maple Leafs,1,Neil Clairmont,LW,NaN,NaN,0,...,1,0,1,0,0,0,0,0,1,0
11429,12246,1963,17,Toronto Maple Leafs,1,Jim McKenny,D,16.0,1979.0,0,...,0,0,1,0,0,0,0,0,1,0


Now, we split the data based upon draft year (taking players from the 2019 draft onward to test the models, using the 1963-2018 drafts to train the model)

In [15]:
test_set = df.loc[df['year'] >= 2019]
test_set.tail()

,id,year,overall_pick,team,league_standing,player,position,age,to_year,amateur_league_0,...,amateur_league_4,amateur_league_5,amateur_league_6,amateur_league_7,nationality_0,nationality_1,nationality_2,nationality_3,nationality_4,nationality_5
869,872,2019,208,St. Louis Blues,12,Vadim Zherenko,G,18.0,NaN,0,...,1,0,0,1,0,0,0,1,1,1
870,881,2019,217,St. Louis Blues,12,Jeremy Michel,LW,18.0,NaN,0,...,1,0,0,0,0,0,0,0,1,0
871,768,2019,104,Columbus Blue Jackets,13,Eric Hjorth,D,18.0,NaN,0,...,0,1,0,1,0,0,0,1,0,1
872,778,2019,114,Columbus Blue Jackets,13,Dimitri Voronkov,LW,19.0,NaN,0,...,0,1,1,0,0,0,0,1,1,1
873,876,2019,212,Columbus Blue Jackets,13,Tyler Angle,C,18.0,NaN,0,...,0,0,1,1,0,0,0,0,1,0


In [16]:
train_set = df.loc[df['year'] < 2019]
train_set.head()

,id,year,overall_pick,team,league_standing,player,position,age,to_year,amateur_league_0,...,amateur_league_4,amateur_league_5,amateur_league_6,amateur_league_7,nationality_0,nationality_1,nationality_2,nationality_3,nationality_4,nationality_5
874,882,2018,1,Buffalo Sabres,31,Rasmus Dahlin,D,18.0,2022.0,0,...,0,1,0,0,0,0,0,1,0,1
875,913,2018,32,Buffalo Sabres,31,Mattias Samuelsson,D,18.0,2022.0,0,...,0,1,0,0,0,0,0,0,1,1
876,975,2018,94,Buffalo Sabres,31,Matej Pekar,C,18.0,NaN,0,...,0,1,1,1,0,0,1,0,0,0
877,998,2018,117,Buffalo Sabres,31,Linus Lindstrand Cronholm,D,18.0,NaN,0,...,0,1,0,1,0,0,0,1,0,1
878,1006,2018,125,Buffalo Sabres,31,Miska Kukkonen,D,18.0,NaN,0,...,1,1,0,0,0,0,0,1,1,0


Additionally, we must split the players from the goalies.

In [17]:
player_train_set = train_set.loc[train_set['position']!='G']
player_test_set = test_set.loc[test_set['position']!='G']

goalie_train_set = train_set.loc[train_set['position']=='G']
goalie_test_set = test_set.loc[test_set['position']=='G']

It is imperative before the dummy variables are created that we make sure that the position column does not have any typos or that equivalent positions but flipped order is equivalent.

In [18]:
def change_pos_value(pos):
  if pos == "LW/C" or pos == "C; LW":
    return "C/LW"
  elif pos == "RW/C" or pos == "C RW" or pos == "C / R":
    return "C/RW"
  elif pos == "D/LW":
    return "LW/D"
  elif pos == "D/RW":
    return "RW/D"
  elif pos == "D/C":
    return "C/D"
  elif pos == "Centr":
    return "C"
  elif pos == "L/RW":
    return "W"
  elif pos == "C/W":
    return "F"
  elif pos == "D/W":
    return "W/D"
  else:
    return pos

position = player_train_set['position']
new_position = position.apply(change_pos_value)
player_train_set.position = new_position

test_pos = player_test_set['position']
new_test_pos = test_pos.apply(change_pos_value)
player_test_set.position = new_test_pos
player_test_set.position.value_counts()

<ipython-input-18-857eaa4e0d72>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_train_set.position = new_position
<ipython-input-18-857eaa4e0d72>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_test_set.position = new_test_pos


D     279
C     229
RW    149
LW    135
Name: position, dtype: int64

Since the number of positions are small, we can simply use one-hot encoding to represent the data in a way that can be fed into the machine learning algorithm. Obviously, this would only affect skaters since all goalies play the same position.

In [19]:
# use one-hot encoding to create "dummy variables" for the SKATER positions
player_train_set = pd.get_dummies(player_train_set, columns=['position'])
player_test_set = pd.get_dummies(player_test_set, columns=['position'])

Now, we must combine these statistics with their respective rookie year statlines using Hockey Reference, the player's name, and their 'to_year' column in the table (since this excludes the 2022-2023 season, then the statistics only go from the 1962-63 season to the 2022-23 season), and this only applies to the training sets in both the skaters and the goalies.

In [20]:
player_train_set

,id,year,overall_pick,team,league_standing,player,age,to_year,amateur_league_0,amateur_league_1,...,position_C/LW,position_C/RW,position_D,position_F,position_LW,position_LW/D,position_RW,position_RW/D,position_W,position_W/D
874,882,2018,1,Buffalo Sabres,31,Rasmus Dahlin,18.0,2022.0,0,0,...,0,0,1,0,0,0,0,0,0,0
875,913,2018,32,Buffalo Sabres,31,Mattias Samuelsson,18.0,2022.0,0,0,...,0,0,1,0,0,0,0,0,0,0
876,975,2018,94,Buffalo Sabres,31,Matej Pekar,18.0,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
877,998,2018,117,Buffalo Sabres,31,Linus Lindstrand Cronholm,18.0,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
878,1006,2018,125,Buffalo Sabres,31,Miska Kukkonen,18.0,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11426,12249,1963,20,New York Rangers,5,Campbell Alleson,NaN,NaN,0,1,...,0,0,1,0,0,0,0,0,0,0
11427,12235,1963,6,Toronto Maple Leafs,1,Walt McKechnie,16.0,1983.0,1,0,...,0,0,0,0,0,0,0,0,0,0
11428,12241,1963,12,Toronto Maple Leafs,1,Neil Clairmont,NaN,NaN,0,1,...,0,0,0,0,1,0,0,0,0,0
11429,12246,1963,17,Toronto Maple Leafs,1,Jim McKenny,16.0,1979.0,0,1,...,0,0,1,0,0,0,0,0,0,0


The next part is relatively tricky, since the qualifications of a "rookie year" can be misconstrued easily. Simply put, this model will contain the last rookie-eligible year of a prospect and will qualify that as their "rookie year", since that is when they would be eligible for the Calder Trophy (the trophy handed out to the best rookie in an NHL season). Simply put, we must consider a prospect's season if (and in this order):
* the NHL player's first season is when they are 25 years of age
* the NHL player played in more than 25 games that season
* the NHL player will have 6 or more games played in the past 2 NHL seasons

Every other year will not be considered and likewise will not be a part of the models' training sets.

In [21]:
import time
#merge the player statistics in the training set
final_pl_season_tbl = pd.DataFrame()

for year in range(1963, 2023):
  #access player stats during the seasonal data and format it with only
  #necessary data for the model
  if year == 2005:  #no player statistics for the 2005 lockout year
    continue
  url = "https://www.hockey-reference.com/leagues/NHL_" + str(year) + "_skaters.html"
  pl_season_tbl = pd.read_html(url)
  pl_season_tbl = pd.DataFrame(pl_season_tbl[0])
  pl_season_tbl = pl_season_tbl.iloc[:, 1:12]
  pl_season_tbl.columns = ['Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'A',
                          'PTS', '+/-', 'PIM', 'PS']

  #concatenate tables as they come
  final_pl_season_tbl = pd.concat([pl_season_tbl, final_pl_season_tbl],
                                  ignore_index=True, axis=0)

  #use this to prevent HTTP errors
  time.sleep(3)

final_pl_season_tbl.reset_index(drop=True, inplace=True)
final_pl_season_tbl = final_pl_season_tbl.sort_values(by=['Player', 'Age'], ascending=[True, True])
final_pl_season_tbl

,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,PIM,PS
5889,A.J. Greer,20,COL,LW,5,0,1,1,-2,4,-0.1
4839,A.J. Greer,21,COL,LW,17,0,3,3,2,29,0.2
3741,A.J. Greer,22,COL,LW,15,1,1,2,-2,14,0.1
1581,A.J. Greer,24,NJD,LW,1,0,0,0,0,7,0.0
409,A.J. Greer,25,NJD,LW,9,1,1,2,0,2,0.2
...,...,...,...,...,...,...,...,...,...,...,...
22511,Žigmund Pálffy,28,LAK,RW,73,38,51,89,22,20,11.1
21397,Žigmund Pálffy,29,LAK,RW,63,32,27,59,5,26,8.1
20269,Žigmund Pálffy,30,LAK,RW,76,37,48,85,22,47,11.4
19088,Žigmund Pálffy,31,LAK,RW,35,16,25,41,18,12,5.7


In [38]:
#!pip install unidecode
from unidecode import unidecode

In [81]:
# iterate over the grouped names and eliminate all but the rookie years
rookie_stats = final_pl_season_tbl
rookie_stats = rookie_stats[rookie_stats['Age'] != "Age"]
rookie_stats = rookie_stats[pd.to_numeric(rookie_stats['Age']) <= 25]
rookie_stats['GP'] = pd.to_numeric(rookie_stats['GP'])
rookie_stats['Age'] = pd.to_numeric(rookie_stats['Age'])
rookie_stats = rookie_stats.reset_index(drop=True)

rows = pd.DataFrame()
season_count=0
found_row = False
for i in rookie_stats.index:
  if season_count == 0:
    # check if more than 25 games played
    if rookie_stats['GP'][i] > 25:
      added = rookie_stats.iloc[i].to_frame().transpose()
      rows = pd.concat([rows, added], ignore_index=True, axis=0)
      found_row = True
    # check if more than 25 games played, at or more than 6 games previous 2 seasons, found_row false
  elif (rookie_stats['GP'][i] > 25 or rookie_stats['GP'][i] \
          + rookie_stats['GP'][i-1] >= 6) and not found_row:
    added = rookie_stats.iloc[i].to_frame().transpose()
    rows = pd.concat([rows, added], ignore_index=True, axis=0)
    found_row = True
  # check if names change, if they do then reset the season_count and found_row
  if (i!=rookie_stats.index[-1]) and (rookie_stats['Player'][i] != rookie_stats['Player'][i+1]):
    season_count=0
    found_row=False
  else:
    season_count = season_count + 1

rookie_stats=rows

# fix the names to ensure that they can match correctly
for i in rookie_stats.index:
  if rookie_stats['Player'][i][-1] == '*':
    rookie_stats['Player'][i] = rookie_stats['Player'][i][:-1]
  rookie_stats['Player'][i] = unidecode(rookie_stats['Player'][i])
#rookie_stats

# join it with the other dataframe and ensure that all the names have values
player_rookie_tbl = player_train_set.merge(rookie_stats, left_on='player', right_on='Player', how='left')
player_rookie_tbl.dropna(subset=['GP', 'G', 'A'], inplace=True)

# make the to_year column the difference between age when drafted and the age
# when they are in their rookie season
player_rookie_tbl['to_year'] = player_rookie_tbl['Age'] - player_rookie_tbl['age']
player_rookie_tbl = player_rookie_tbl.rename(columns={'to_year':'yrs_til_rookie'})
player_rookie_tbl = player_rookie_tbl.drop(columns=["Age", "age", "Pos", "Tm", "Player"])
player_rookie_tbl

,id,year,overall_pick,team,league_standing,player,yrs_til_rookie,amateur_league_0,amateur_league_1,amateur_league_2,...,position_RW/D,position_W,position_W/D,GP,G,A,PTS,+/-,PIM,PS
0,882,2018,1,Buffalo Sabres,31,Rasmus Dahlin,0.0,0,0,0,...,0,0,0,82,9,35,44,-13,34,6.6
1,913,2018,32,Buffalo Sabres,31,Mattias Samuelsson,3.0,0,0,0,...,0,0,0,42,0,10,10,-10,16,1.2
6,883,2018,2,Carolina Hurricanes,21,Andrei Svechnikov,0.0,0,0,0,...,0,0,0,82,20,17,37,0,62,3.9
11,884,2018,3,Montreal Canadiens,28,Jesperi Kotkaniemi,0.0,0,0,0,...,0,0,0,79,11,23,34,1,26,2.6
12,916,2018,35,Montreal Canadiens,28,Jesse Ylonen,4.0,0,0,1,...,0,0,0,14,2,3,5,2,2,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9477,12230,1963,1,Montreal Canadiens,3,Garry Monahan,6.0,0,1,0,...,0,0,0,3,0,0,0,0,0,-0.1
9481,12231,1963,2,Detroit Red Wings,4,Pete Mahovlich,4.0,0,1,0,...,0,0,0,34,1,3,4,-8,16,-0.7
9491,12235,1963,6,Toronto Maple Leafs,1,Walt McKechnie,5.0,1,0,0,...,0,0,0,58,5,9,14,-10,22,0.0
9493,12246,1963,17,Toronto Maple Leafs,1,Jim McKenny,4.0,0,1,1,...,0,0,0,6,1,0,1,1,0,0.4


Now with the data preprocessed, we can perform some significance tests on it and then split it into a features and output matrix. However, we must first do a similar method to combine the rookie stats with the draft information from before.

In [85]:
#merge the goalie statistics in the training set, similar to how we did it above
#merge the player statistics in the training set
final_g_season_tbl = pd.DataFrame()

for year in range(1963, 2023):
  #access player stats during the seasonal data and format it with only
  #necessary data for the model
  if year == 2005:  #no player statistics for the 2005 lockout year
    continue
  url = "https://www.hockey-reference.com/leagues/NHL_" + str(year) + "_goalies.html"
  g_season_tbl = pd.read_html(url)
  g_season_tbl = pd.DataFrame(g_season_tbl[0])
  g_season_tbl = g_season_tbl.iloc[:, 1:]
  g_season_tbl.columns = g_season_tbl.columns.droplevel()
  g_season_tbl = g_season_tbl.drop(columns=["L", "T/O", "GA", "SA", "SV", "GS"])
  g_season_tbl = g_season_tbl.iloc[:, 0:7]

  #concatenate tables as they come
  final_g_season_tbl = pd.concat([g_season_tbl, final_g_season_tbl],
                                  ignore_index=True, axis=0)

  #use this to prevent HTTP errors
  time.sleep(3)

final_g_season_tbl.reset_index(drop=True, inplace=True)
final_g_season_tbl = final_g_season_tbl.sort_values(by=['Player', 'Age'], ascending=[True, True])
final_g_season_tbl

,Player,Age,Tm,GP,W,SV%,GAA
596,Aaron Dell,27,SJS,20,11,.931,2.00
476,Aaron Dell,28,SJS,29,15,.914,2.64
368,Aaron Dell,29,SJS,25,10,.886,3.17
262,Aaron Dell,30,SJS,33,12,.907,3.01
150,Aaron Dell,31,NJD,7,1,.857,4.14
...,...,...,...,...,...,...,...
2543,Éric Fichaud,21,NYI,34,9,.899,3.10
2450,Éric Fichaud,22,NYI,17,3,.905,2.97
2349,Éric Fichaud,23,NSH,9,0,.895,3.22
2247,Éric Fichaud,24,CAR,9,3,.883,2.94


In [86]:
# iterate over the grouped names and eliminate all but the rookie years
rookie_stats = pd.DataFrame()
rookie_stats = final_g_season_tbl
rookie_stats = rookie_stats[rookie_stats['Age'] != "Age"]
rookie_stats = rookie_stats[pd.to_numeric(rookie_stats['Age']) <= 25]
rookie_stats['GP'] = pd.to_numeric(rookie_stats['GP'])
rookie_stats['Age'] = pd.to_numeric(rookie_stats['Age'])
rookie_stats = rookie_stats.reset_index(drop=True)

rows = pd.DataFrame()
season_count=0
found_row = False
for i in rookie_stats.index:
  if season_count == 0:
    # check if more than 25 games played
    if rookie_stats['GP'][i] > 25:
      added = rookie_stats.iloc[i].to_frame().transpose()
      rows = pd.concat([rows, added], ignore_index=True, axis=0)
      found_row = True
    # check if more than 25 games played, at or more than 6 games previous 2 seasons, found_row false
  elif (rookie_stats['GP'][i] > 25 or rookie_stats['GP'][i] \
          + rookie_stats['GP'][i-1] >= 6) and not found_row:
    added = rookie_stats.iloc[i].to_frame().transpose()
    rows = pd.concat([rows, added], ignore_index=True, axis=0)
    found_row = True
  # check if names change, if they do then reset the season_count and found_row
  if (i!=rookie_stats.index[-1]) and (rookie_stats['Player'][i] != rookie_stats['Player'][i+1]):
    season_count=0
    found_row=False
  else:
    season_count = season_count + 1

rookie_stats=rows

# fix the names to ensure that they can match correctly
for i in rookie_stats.index:
  if rookie_stats['Player'][i][-1] == '*':
    rookie_stats['Player'][i] = rookie_stats['Player'][i][:-1]
  rookie_stats['Player'][i] = unidecode(rookie_stats['Player'][i])
#rookie_stats

# join it with the other dataframe and ensure that all the names have values
goalie_rookie_tbl = goalie_train_set.merge(rookie_stats, left_on='player', right_on='Player', how='left')
goalie_rookie_tbl.dropna(subset=['GP', 'Player', 'Tm'], inplace=True)

# make the to_year column the difference between age when drafted and the age
# when they are in their rookie season
goalie_rookie_tbl['to_year'] = goalie_rookie_tbl['Age'] - goalie_rookie_tbl['age']
goalie_rookie_tbl = goalie_rookie_tbl.rename(columns={'to_year':'yrs_til_rookie'})
goalie_rookie_tbl

,id,year,overall_pick,team,league_standing,player,position,age,yrs_til_rookie,amateur_league_0,...,nationality_3,nationality_4,nationality_5,Player,Age,Tm,GP,W,SV%,GAA
2,995,2018,114,Arizona Coyotes,29,Ivan Prosvetov,G,19.0,3.0,0,...,1,1,1,Ivan Prosvetov,22,ARI,3,0,.875,4.22
31,1124,2017,26,Dallas Stars,24,Jake Oettinger,G,18.0,4.0,0,...,0,1,1,Jake Oettinger,22,DAL,29,11,.911,2.36
35,1152,2017,54,Buffalo Sabres,26,Ukko-Pekka Luukkonen,G,18.0,4.0,0,...,1,1,0,Ukko-Pekka Luukkonen,22,BUF,9,2,.917,2.74
40,1209,2017,111,Boston Bruins,13,Jeremy Swayman,G,18.0,5.0,0,...,0,1,1,Jeremy Swayman,23,BOS,41,23,.914,2.41
41,1176,2017,78,Edmonton Oilers,7,Stuart Skinner,G,18.0,5.0,0,...,0,1,0,Stuart Skinner,23,EDM,13,6,.913,2.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,12054,1969,10,Detroit Red Wings,3,Jim Rutherford,G,20.0,1.0,1,...,0,1,0,Jim Rutherford,21,DET,29,7,.876,3.77
1055,12129,1968,1,Montreal Canadiens,6,Michel Plasse,G,20.0,4.0,1,...,0,1,0,Michel Plasse,24,MTL,17,11,.912,2.58
1057,12134,1968,6,St. Louis Blues,11,Gary Edwards,G,20.0,4.0,1,...,0,1,0,Gary Edwards,24,LAK,44,13,.898,3.60
1058,12175,1966,5,Montreal Canadiens,1,Phil Myre,G,17.0,5.0,1,...,0,1,0,Phil Myre,22,MTL,30,13,.905,3.12


# Phase 2: Finding Significance using ANOVA F-test

Before the models can be created, significance has to be tested among the variables.

For the players, the models will be predicting:

*   games played
*   goals
*   assists
*   points
*   penalty minutes
*   point shares (estimation of how many of team's points a player contributes to)


For the goalies, the models will be predicting:

*   games played
*   wins
*   save percentage
*   goals against average (GAA)

An ANOVA F-test can help determine correlations between the features of the train set and the variables that will be predicted on

# Phase 3: Creating the Models

# Phase 4: Training the Models and Tuning the Hyperparameters

# Phase 5: Analysis of Model Performance

From here on out, the work in the iPYNB file is complete, now it is time to create the CLI for the program and "polish" the code to make it look and perform more professionally. See you then! 😁